<a href="https://colab.research.google.com/github/hevertonvalerio/Analise_Exploratoria/blob/main/projeto_aplicado_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

### ANALISE E FILTRAGEM DOS DADOS

In [ ]:
df_info = pd.read_csv('/content/drive/MyDrive/Mackenzie/Projeto Aplicado 3/Course_info.csv')

In [ ]:
df_info.head()

,id,title,is_paid,price,headline,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,published_time,last_update_date,category,subcategory,topic,language,course_url,instructor_name,instructor_url
0,4715.0,Online Vegan Vegetarian Cooking School,True,24.99,Learn to cook delicious vegan recipes. Filmed ...,2231.0,3.75,134.0,42.0,37.0,1268.0,2010-08-05T22:06:13Z,2020-11-06,Lifestyle,Food & Beverage,Vegan Cooking,English,/course/vegan-vegetarian-cooking-school/,Angela Poch,/user/angelapoch/
1,1769.0,The Lean Startup Talk at Stanford E-Corner,False,0.00,Debunking Myths of Entrepreneurship A startup ...,26474.0,4.50,709.0,112.0,9.0,88.0,2010-01-12T18:09:46Z,NaN,Business,Entrepreneurship,Lean Startup,English,/course/the-lean-startup-debunking-myths-of-en...,Eric Ries,/user/ericries/
2,5664.0,"How To Become a Vegan, Vegetarian, or Flexitarian",True,19.99,Get the tools you need for a lifestyle change ...,1713.0,4.40,41.0,13.0,14.0,82.0,2010-10-13T18:07:17Z,2019-10-09,Lifestyle,Other Lifestyle,Vegan Cooking,English,/course/see-my-personal-motivation-for-becomin...,Angela Poch,/user/angelapoch/
3,7723.0,How to Train a Puppy,True,199.99,Train your puppy the right way with Dr. Ian Du...,4988.0,4.80,395.0,88.0,36.0,1511.0,2011-06-20T20:08:38Z,2016-01-13,Lifestyle,Pet Care & Training,Pet Training,English,/course/complete-dunbar-collection/,Ian Dunbar,/user/ian-dunbar/
4,8157.0,Web Design from the Ground Up,True,159.99,Learn web design online: Everything you need t...,1266.0,4.75,38.0,12.0,38.0,569.0,2011-06-23T18:31:20Z,NaN,Design,Web Design,Web Design,English,/course/web-design-from-the-ground-up/,E Learning Lab,/user/edwin-ang-2/


In [ ]:
df_info.shape

(209734, 20)

In [ ]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209734 entries, 0 to 209733
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  209734 non-null  float64
 1   title               209734 non-null  object 
 2   is_paid             209734 non-null  bool   
 3   price               209734 non-null  float64
 4   headline            209703 non-null  object 
 5   num_subscribers     209734 non-null  float64
 6   avg_rating          209734 non-null  float64
 7   num_reviews         209734 non-null  float64
 8   num_comments        209734 non-null  float64
 9   num_lectures        209734 non-null  float64
 10  content_length_min  209734 non-null  float64
 11  published_time      209734 non-null  object 
 12  last_update_date    209597 non-null  object 
 13  category            209734 non-null  object 
 14  subcategory         209734 non-null  object 
 15  topic               208776 non-nul

In [ ]:
df_info.columns

Index(['id', 'title', 'is_paid', 'price', 'headline', 'num_subscribers',
       'avg_rating', 'num_reviews', 'num_comments', 'num_lectures',
       'content_length_min', 'published_time', 'last_update_date', 'category',
       'subcategory', 'topic', 'language', 'course_url', 'instructor_name',
       'instructor_url'],
      dtype='object')

In [ ]:
df_info['language'].unique()

array(['English', 'Spanish', 'Turkish', 'Simplified Chinese', 'Arabic',
       'Portuguese', 'Italian', 'Serbian', 'Afrikaans', 'French',
       'Slovak', 'Japanese', 'Hebrew', 'Traditional Chinese', 'Estonian',
       'German', 'Russian', 'Finnish', 'Dutch', 'Hungarian', 'Swedish',
       'Norwegian', 'Thai', 'Bulgarian', 'Polish', 'Urdu', 'Croatian',
       'Marathi', 'Danish', 'Greek', 'Hindi', 'Indonesian', 'Azeri',
       'Vietnamese', 'Bengali', 'Persian', 'Malay', 'Korean', 'Filipino',
       'Romanian', 'Catalan', 'Czech', 'Albanian', 'Telugu', 'Latvian',
       'Ukrainian', 'Tamil', 'Somali', 'Burmese', 'Kazakh', 'Tatar',
       'Kurdish', 'Aymara', 'Malayalam', 'Uzbek', 'Georgian',
       'Lithuanian', 'Gujarati', 'Pashto', 'Haitian', 'Nepali',
       'Armenian', 'Punjabi', 'Swahili', 'Mongolian', 'Slovenian', 'Zulu',
       'Kannada', 'Guaraní', 'Quechua', 'Macedonian', 'Khmer', 'Frisian',
       'Welsh', 'Irish', 'Basque', 'Xhosa', 'Tajik', 'Faroese'],
      dtype=object)

In [ ]:
df = df_info[['id', 'title', 'price', 'avg_rating', 'headline']].copy()

In [ ]:
df = df_info[df_info['language'].isin(['English', 'Spanish', 'Portuguese'])].copy()

In [ ]:
df.head()

,id,title,is_paid,price,headline,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,published_time,last_update_date,category,subcategory,topic,language,course_url,instructor_name,instructor_url
0,4715.0,Online Vegan Vegetarian Cooking School,True,24.99,Learn to cook delicious vegan recipes. Filmed ...,2231.0,3.75,134.0,42.0,37.0,1268.0,2010-08-05T22:06:13Z,2020-11-06,Lifestyle,Food & Beverage,Vegan Cooking,English,/course/vegan-vegetarian-cooking-school/,Angela Poch,/user/angelapoch/
1,1769.0,The Lean Startup Talk at Stanford E-Corner,False,0.00,Debunking Myths of Entrepreneurship A startup ...,26474.0,4.50,709.0,112.0,9.0,88.0,2010-01-12T18:09:46Z,NaN,Business,Entrepreneurship,Lean Startup,English,/course/the-lean-startup-debunking-myths-of-en...,Eric Ries,/user/ericries/
2,5664.0,"How To Become a Vegan, Vegetarian, or Flexitarian",True,19.99,Get the tools you need for a lifestyle change ...,1713.0,4.40,41.0,13.0,14.0,82.0,2010-10-13T18:07:17Z,2019-10-09,Lifestyle,Other Lifestyle,Vegan Cooking,English,/course/see-my-personal-motivation-for-becomin...,Angela Poch,/user/angelapoch/
3,7723.0,How to Train a Puppy,True,199.99,Train your puppy the right way with Dr. Ian Du...,4988.0,4.80,395.0,88.0,36.0,1511.0,2011-06-20T20:08:38Z,2016-01-13,Lifestyle,Pet Care & Training,Pet Training,English,/course/complete-dunbar-collection/,Ian Dunbar,/user/ian-dunbar/
4,8157.0,Web Design from the Ground Up,True,159.99,Learn web design online: Everything you need t...,1266.0,4.75,38.0,12.0,38.0,569.0,2011-06-23T18:31:20Z,NaN,Design,Web Design,Web Design,English,/course/web-design-from-the-ground-up/,E Learning Lab,/user/edwin-ang-2/


In [ ]:
df.shape

(159820, 20)

In [ ]:
df = df_info[(df_info['price'] <= 20) & (df_info['avg_rating'] >= 4.5)].copy()

In [ ]:
df.shape

(26379, 20)

In [ ]:
df = df.dropna(subset=['headline']).copy()

In [ ]:
df.shape

(26373, 20)

In [ ]:
df_final = df.copy()

In [ ]:
df_final['headline'] = df_final['headline'].str.lower()

In [ ]:
df_final.to_csv('df_final.csv', index=False)

# TESTE DO MODELO

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Mackenzie/Projeto Aplicado 3/df_final.csv')

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
headlines = df['headline'].tolist()

In [ ]:
# embeddings = model.encode(headlines, show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/825 [00:00<?, ?it/s]

In [ ]:
embeddings_path = '/content/drive/MyDrive/Mackenzie/Projeto Aplicado 3/udemy_courses_headline_embeddings.npy'

In [ ]:
# np.save(embeddings_path, embeddings)

# Recommendation - Tests

---



In [ ]:
embeddings = np.load(embeddings_path)

In [ ]:
df = df[['id', 'title', 'price', 'avg_rating', 'course_url', 'instructor_name', 'headline']].copy()

In [ ]:
def recommendation(skills, top_k=100, results=10):
    query = ', '.join(skills)
    query_embedding = model.encode([query])

    similarities = cosine_similarity(query_embedding, embeddings)[0]

    top_k_indices = np.argsort(similarities)[-top_k:][::-1]

    candidate_course = df.iloc[top_k_indices].copy()
    candidate_course['similarity'] = similarities[top_k_indices]
    candidate_course = candidate_course.sort_values('similarity', ascending=False)

    return candidate_course.head(results)

In [ ]:
skills = ['data science', 'python', 'machine learning']

In [ ]:
recommendation(skills, 50, 20)

,id,title,price,avg_rating,course_url,instructor_name,headline,similarity
17686,4129008.0,Data Science_Machine Learning with Python Prog...,19.99,5.000000,/course/data-science_machine-learning-with-pyt...,Mythili Eragamreddy,data science_machine learning with python prog...,0.877996
15087,3857472.0,Data Science – End 2 End Beginners Course Part 1,19.99,4.666666,/course/datascience-e2e-beginnerscourse-machin...,Nitin Singhal,"machine learning & data analytics- python, pan...",0.854361
13282,3642544.0,Python for Data Science,0.00,4.550000,/course/python-for-data-science-t/,Saima Aziz,learn python for data science,0.830209
15683,3914646.0,Data Science Bootcamp 2022: 5 Data Science Pro...,19.99,4.515151,/course/data-science-bootcamp-with-python/,Data Is Good Academy,data science and machine learning masterclass ...,0.825248
24495,4725264.0,Data Science and Machine Learning in Python: L...,19.99,4.852941,/course/data-science-ml-python/,Escape Velocity Labs,master the most popular data science and machi...,0.820468
8825,2901794.0,"Data Science with Analogies, Algorithms and So...",0.00,4.500000,/course/data-science-with-analogies-algorithms...,Ajay Dhruv,"machine learning, data mining, data science, d...",0.805630
26238,4890746.0,Deep Dive into Artificial Intelligence - The M...,7.00,5.000000,/course/deep-dive-into-artificial-intelligence...,Skillcart E-learning,artificial intelligence | data science with py...,0.798842
12687,3571377.0,"Data Science with ML, AI, GUI & VUI with Libra...",1.00,4.500000,/course/data-science-with-ml-ai-gui-vui-with-l...,Piyushh n Dave,"data science, machine learning , artificial in...",0.798464
20081,4360142.0,"Data science ,Analytics & AI Real world Projec...",1.00,4.800000,/course/data-science-analytics-ai-real-world-p...,Shan Singh,"master ai , data analytics , machine learning ...",0.793418
20843,4418410.0,Practical Data Science using Python,4.00,4.800000,/course/practical-data-science-using-python-md/,Manas Dasgupta,"apply data science using python, statistical t...",0.776751
